In [2]:
import networkx as nx

G = nx.read_gpickle('./data/padded_train/Malware/67bc86683b9b63d6c22f238c54c15e8e3cc4e80a14e0689363e7c5cb88941598.gpickle')

In [3]:
G.nodes[99]

{'bName': '0x80482cc[10]',
 'Start': 134513356,
 'End': 134513364,
 'Next': [134513376, 134513366],
 'Ins': [(134513356, ['mov', 'ax, word ptr [ebp - 0x22]']),
  (134513360, ['cmp', 'ax, 3']),
  (134513364, ['je', '0x80482e0'])],
 'Prev': [134513350],
 'feat': [1.0, 0.0, 1.0, 0.0, 3.0, 1.0, 2.0, 0.2572375658618493]}

In [4]:
G.edges(98)

OutMultiEdgeDataView([(98, 99), (98, 275)])

In [5]:
weakly_connected_subgraphs = nx.weakly_connected_components(G)

In [6]:
weakly_subgraphs = [subgraph for subgraph in weakly_connected_subgraphs if len(subgraph) > 2]

In [7]:
len(weakly_subgraphs[0])

305

In [8]:
H = nx.read_gpickle('./interpretability_results/Malware/67bc86683b9b63d6c22f238c54c15e8e3cc4e80a14e0689363e7c5cb88941598./subgraph_10percent.gpickle')

In [9]:
weakly_connected_subgraphs_H = nx.weakly_connected_components(H)

In [10]:
weakly_subgraphs_H = [subgraph for subgraph in weakly_connected_subgraphs_H if len(subgraph) >= 2]

In [39]:
weakly_subgraphs_H

[{47, 49, 51, 54, 55, 57, 58, 60, 271, 272},
 {98, 99, 100, 101, 102, 103, 118},
 {144, 147, 148, 151, 152, 154, 279, 281},
 {162,
  165,
  166,
  167,
  168,
  169,
  170,
  171,
  172,
  173,
  174,
  175,
  176,
  177,
  178,
  179,
  184,
  185,
  186,
  187,
  196,
  197,
  200,
  203,
  210,
  211,
  212,
  221,
  223,
  224,
  285}]

In [103]:
G.nodes()

NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 

In [304]:
summary_dir = '/mnt/bigDisk/leon/rtools/CFGs/x86/'
filename = '67bc86683b9b63d6c22f238c54c15e8e3cc4e80a14e0689363e7c5cb88941598'
ACFG = nx.read_gpickle(summary_dir + filename + '.pickle')
ACFG = nx.MultiDiGraph(ACFG)

In [305]:
def Caculate_ACFG_boundary(ACFG): 
    ACFG_Start = int(ACFG.nodes[0]['x'][0][0].split('[')[0], 16)
    ACFG_End = int(ACFG.nodes[0]['x'][-1][0].split('[')[0], 16)
    for node, data in ACFG.nodes(data=True):
        if "temp" in str(node):
            continue
        Start = int(data['x'][0][0].split('[')[0], 16)
        End = int(data['x'][-1][0].split('[')[0], 16)
        if Start < ACFG_Start:
            ACFG_Start = Start
        if ACFG_End < End:
            ACFG_End = End
    if ACFG_End % 2:
        ACFG_End += 1
    return ACFG_Start, ACFG_End

In [306]:
def Partition(node, addr, part_cnt):
    if len(ACFG.nodes[node]['x']) < 2:
        return
    part_ins = ACFG.nodes[node]['x'][:-1]
    ACFG.nodes[node]['x'] = [[ACFG.nodes[node]['x'][0][0]] + ACFG.nodes[node]['x'][-1][1:]]
    in_nodes = [edge[0] for edge in ACFG.in_edges(node)]

    edges_to_remove = list(ACFG.in_edges(node))
    ACFG.remove_edges_from(edges_to_remove)
    FirstNode = True
    for ins in part_ins:
        # newnode = 'Partition_Node_' + str(part_cnt)
        newnode = part_cnt
        ins[0] = str(hex(addr))
        addr += 4
        newins = [ins, [str(hex(addr)), 'jmp ' + str(hex(addr + 4))]]
        addr += 4
        ACFG.add_node(newnode, x = newins)
        for in_node in in_nodes:
            ACFG.add_edge(in_node, newnode)
            if FirstNode: 
                if ACFG.nodes[in_node]['x'][-1][1].split()[-1] == ACFG.nodes[node]['x'][0][0]:
                    ACFG.nodes[in_node]['x'][-1][1].split()[-1] = ACFG.nodes[newnode]['x'][0][0]
                else:
                    ACFG.nodes[in_node]['x'] += [[str(hex(int(ACFG.nodes[in_node]['x'][-1][0], 16) + 4 )), 'jmp ' + ACFG.nodes[newnode]['x'][0][0]]]
        in_nodes = [newnode]
        part_cnt += 1
        FirstNode = False
    ACFG.nodes[newnode]['x'][-1][1] = 'jmp ' + str(ACFG.nodes[99]['x'][0][0])

    ACFG.add_edge(in_nodes[0], node)    
    return addr, part_cnt

In [307]:
part_cnt = len(ACFG.nodes)
print('start', part_cnt)
ACFG_Start, ACFG_End = Caculate_ACFG_boundary(ACFG)
print(ACFG_Start, ACFG_End)
addr = ACFG_End + 4
addr, part_cnt = Partition(99, addr, part_cnt)
print('to 99:', part_cnt)
addr, part_cnt = Partition(98, addr, part_cnt)
print('to 98:', part_cnt)
addr, part_cnt = Partition(55, addr, part_cnt)
print('to 55:', part_cnt)
addr, part_cnt = Partition(171, addr, part_cnt)
print('to 177:', part_cnt)
addr, part_cnt = Partition(166, addr, part_cnt)
print('to 166:', part_cnt)
ACFG_Start, ACFG_End = Caculate_ACFG_boundary(ACFG)
print(ACFG_Start, ACFG_End)

start 305
134512756 137363598
to 99: 307
to 98: 311
to 55: 321
to 177: 326
to 166: 331
134512756 137363806


In [291]:
from Generate_poison_data import Generate_ACFG_Node_Attributes, Append_temp_nodes
ACFG = Generate_ACFG_Node_Attributes(ACFG)
if ACFG.number_of_nodes() < 4096:
    ACFG = Append_temp_nodes(ACFG)
nx.write_gpickle(ACFG, '/home/ycchiu/git-rep/CFGE2BA/CFGExplainer/Partition_samples/padded_train/' + filename + ".gpickle")      

In [248]:
print(ACFG.nodes[98]['x'])
print(ACFG.edges(98))
print(ACFG.in_edges(98))

[['0x80482ba', 'jmp 0x8300092']]
[(98, 275), (98, 'Partition_Node_0')]
[('Partition_Node_5', 98)]


In [239]:
print(ACFG.nodes['Partition_Node_0']['x'])
print(ACFG.edges('Partition_Node_0'))
print(ACFG.in_edges('Partition_Node_0'))

[['0x8300092', 'mov ax, word ptr [ebp - 0x22]'], ['0x8300096', 'jmp 0x830009a']]
[('Partition_Node_0', 'Partition_Node_1')]
[(98, 'Partition_Node_0')]


In [236]:
print(ACFG.nodes['Partition_Node_1']['x'])
print(ACFG.edges('Partition_Node_1'))
print(ACFG.in_edges('Partition_Node_1'))

[['0x830009a', 'cmp ax, 3'], ['0x830009e', 'jmp 0x80482cc']]
[('Partition_Node_1', 99)]
[('Partition_Node_0', 'Partition_Node_1')]


In [235]:
print(ACFG.nodes[99]['x'])
print(ACFG.edges(99))
print(ACFG.in_edges(99))

[['0x80482cc', 'je 0x80482e0']]
[(99, 100), (99, 101)]
[('Partition_Node_1', 99)]


In [296]:
G = nx.read_gpickle('/home/ycchiu/git-rep/CFGE2BA/CFGExplainer/interpretability_results/Malware/67bc86683b9b63d6c22f238c54c15e8e3cc4e80a14e0689363e7c5cb88941598./subgraph_10percent.gpickle')
weakly_connected_subgraphs_G = nx.weakly_connected_components(G)

In [297]:
weakly_connected_subgraphs_G

<generator object weakly_connected_components at 0x7f728549b6d0>

In [298]:
weakly_subgraphs_G = [subgraph for subgraph in weakly_connected_subgraphs_G if len(subgraph) >= 2]

In [310]:
weakly_subgraphs_G

[{55, 57, 60, 272, 320},
 {99, 100, 101, 102, 103, 115, 116, 117, 118, 121, 306},
 {130, 134, 136, 277},
 {144, 147, 148, 151, 152, 154, 279, 281},
 {171,
  174,
  175,
  176,
  177,
  178,
  179,
  184,
  185,
  186,
  187,
  196,
  197,
  200,
  202,
  203,
  210,
  211,
  212,
  218,
  219,
  220,
  221,
  223,
  224,
  228,
  229,
  232,
  283,
  285,
  325}]

In [315]:
import pickle

with open("/home/ycchiu/git-rep/CFGE2BA/CFGExplainer/interpretability_results/Malware/67bc86683b9b63d6c22f238c54c15e8e3cc4e80a14e0689363e7c5cb88941598/top_blocks.pickle", "rb") as f:
    data = pickle.load(f)

print(data)


[99, 306, 100, 148, 57, 211, 102, 185, 218, 174, 55, 60, 210, 151, 279, 272, 320, 197, 224, 221, 171, 177, 118, 179, 203, 154, 144, 147, 281, 103, 101, 196, 223, 285, 220, 187, 176, 212, 178, 202, 283, 325, 134, 277, 175, 186, 152, 130, 219, 200, 184, 136, 115, 232, 117, 121, 140, 155, 192, 209, 216, 229, 255, 258, 287, 228, 116, 127, 131, 133, 278, 143, 260, 225, 205, 259, 324, 124, 126, 122, 180, 198, 230, 233, 195, 207, 217, 227, 238, 262, 280, 145, 206, 226, 234, 237, 261, 282, 284, 286, 254, 215, 191, 172, 183, 123, 235, 239, 128, 138, 146, 129, 181, 199, 231, 240, 135, 142, 159, 160, 189, 194, 204, 214, 242, 244, 247, 253, 132, 149, 150, 182, 241, 208, 236, 28, 52, 252, 213, 257, 188, 222, 201, 137, 95, 157, 164, 139, 243, 156, 248, 32, 96, 98, 161, 165, 170, 37, 49, 63, 67, 71, 250, 34, 249, 275, 74, 104, 61, 270, 319, 158, 54, 162, 166, 169, 47, 73, 58, 329, 72, 44, 330, 39, 56, 97, 251, 3, 7, 76, 106, 110, 269, 0, 11, 40, 59, 114, 256, 173, 310, 51, 75, 141, 48, 64, 193, 26, 5